References
- https://docs.llamaindex.ai/en/latest/examples/query_engine/knowledge_graph_query_engine.html#
- https://docs.llamaindex.ai/en/latest/examples/index_structs/knowledge_graph/KuzuGraphDemo.html#
- https://github.com/sroecker/LLM_AppDev-HandsOn

In [1]:
from IPython.display import Markdown, display

In [2]:
import logging
import sys

logging.basicConfig(
    stream=sys.stdout, level=logging.INFO
)  # logging.DEBUG for more verbose output

## Load model from Ollama

In [3]:
from llama_index.llms import Ollama

In [4]:
OLLAMA_HOST = 'localhost'
OLLAMA_MODEL = 'mistral'
llm = Ollama(model=OLLAMA_MODEL, base_url="http://"+OLLAMA_HOST+":11434")

In [5]:
from llama_index import ServiceContext
service_context = ServiceContext.from_defaults(llm=llm, embed_model="local")

/Users/louis.guitton/workspace/mlops-talk-llm-kg/venv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Prepare for Kuzu

In [6]:
from llama_index import (
    KnowledgeGraphIndex,
    ServiceContext,
    SimpleDirectoryReader,
)
from llama_index.graph_stores import KuzuGraphStore

In [7]:
import shutil

shutil.rmtree("./test1", ignore_errors=True)
shutil.rmtree("./test2", ignore_errors=True)
shutil.rmtree("./test3", ignore_errors=True)

In [8]:
import kuzu

db = kuzu.Database("test1")

In [9]:
graph_store = KuzuGraphStore(db)

In [10]:
from llama_index.storage.storage_context import StorageContext
storage_context = StorageContext.from_defaults(graph_store=graph_store)

## Build the Knowledge Graph with LlamaIndex

In [11]:
from llama_index import download_loader

WikipediaReader = download_loader("WikipediaReader")

loader = WikipediaReader()

documents = loader.load_data(
    pages=["Guardians of the Galaxy Vol. 3"], auto_suggest=False
)

  Using cached wikipedia-1.4.0.tar.gz (27 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
Using legacy 'setup.py install' for wikipedia, since package 'wheel' is not installed.
  Running setup.py install for wikipedia: started
  Running setup.py install for wikipedia: finished with status 'done'


You should consider upgrading via the '/Users/louis.guitton/workspace/mlops-talk-llm-kg/venv/bin/python -m pip install --upgrade pip' command.


In [12]:
space_name = "llamaindex"
edge_types, rel_prop_names = ["relationship"], [
    "relationship"
]  # default, could be omit if create from an empty kg
tags = ["entity"]  # default, could be omit if create from an empty kg

In [13]:
kg_index = KnowledgeGraphIndex.from_documents(
    documents,
    storage_context=storage_context,
    max_triplets_per_chunk=10,
    service_context=service_context,
    space_name=space_name,
    edge_types=edge_types,
    rel_prop_names=rel_prop_names,
    tags=tags,
    include_embeddings=True,
)

 (Guardians of the Galaxy Vol. 3, is, American superhero film)
(Guardians of the Galaxy Vol. 3, based on, Marvel Comics superhero team Guardians of the Galaxy)
(Guardians of the Galaxy Vol. 3, produced by, Marvel Studios)
(Guardians of the Galaxy Vol. 3, distributed by, Walt Disney Studios Motion Pictures)
(Guardians of the Galaxy Vol. 3, sequel to, Guardians of the Galaxy (2014))
(Guardians of the Galaxy Vol. 3, sequel to, Guardians of the Galaxy Vol. 2 (2017))
(Guardians of the Galaxy Vol. 3, written and directed by, James Gunn)
(Chris Pratt, plays character in, Guardians of the Galaxy Vol. 3)
(Zoe Saldaña, plays character in, Guardians of the Galaxy Vol. 3)
(Dave Bautista, plays character in, Guardians of the Galaxy Vol. 3)
(Karen Gillan, plays character in, Guardians of the Galaxy Vol. 3)
(Pom Klementieff, plays character in, Guardians of the Galaxy Vol. 3)
(Vin Diesel, provides voice for character in, Guardians of the Galaxy Vol. 3)
(Bradley Cooper, provides voice for character in

RuntimeError: Parameters must be a dict

## Perform Graph RAG Query

Finally, let's demo how to do Graph RAG towards an existing Knowledge Graph.

All we need to do is to use `RetrieverQueryEngine` and configure the retriver of it to be `KnowledgeGraphRAGRetriever`.

The `KnowledgeGraphRAGRetriever` performs the following steps:

- Search related Entities of the quesion/task
- Get SubGraph of those Entities (default 2-depth) from the KG
- Build Context based on the SubGraph

Please note, the way to Search related Entities could be either Keyword extraction based or Embedding based, which is controlled by argument `retriever_mode` of the `KnowledgeGraphRAGRetriever`, and supported options are:
- "keyword"
- "embedding"(not yet implemented)
- "keyword_embedding"(not yet implemented)

Here is the example on how to use `RetrieverQueryEngine` and `KnowledgeGraphRAGRetriever`:

In [ ]:
from llama_index.query_engine import RetrieverQueryEngine
from llama_index.retrievers import KnowledgeGraphRAGRetriever

graph_rag_retriever = KnowledgeGraphRAGRetriever(
    storage_context=storage_context,
    service_context=service_context,
    llm=llm,
    verbose=True,
)

query_engine = RetrieverQueryEngine.from_args(
    graph_rag_retriever, service_context=service_context
)

In [ ]:
response = query_engine.query(
    "Tell me about Peter Quill?",
)
display(Markdown(f"<b>{response}</b>"))

In [ ]:
response = await query_engine.aquery(
    "Tell me about Peter Quill?",
)
display(Markdown(f"<b>{response}</b>"))